In [ ]:
# Read the input
with open('input.txt', 'r') as f:
    grid = [line.rstrip('\n') for line in f.readlines()]

# Find the starting position
start_col = None
for col in range(len(grid[0])):
    if grid[0][col] == 'S':
        start_col = col
        break

print(f"Grid dimensions: {len(grid)} rows x {len(grid[0])} columns")
print(f"Starting column: {start_col}")
print(f"\nFirst few rows:")
for i in range(min(5, len(grid))):
    print(grid[i])

## Part 1: Classical Beam Splitting

### Algorithm
1. Start with a beam at column `start_col`, row 0
2. For each active beam:
   - Move it down until it hits a splitter `^` or exits the grid
   - If it hits a splitter, increment the split count and create two new beams (left and right)
3. Continue until no active beams remain

We need to track which beams we've already processed to avoid infinite loops (beams converging and re-splitting).

In [ ]:
def solve_part1(grid, start_col):
    """Count the number of unique splitters hit in the classical interpretation."""
    splitters_hit = set()
    beams = [(1, start_col)]  # Start from row 1 (below 'S')
    processed = set()
    
    while beams:
        row, col = beams.pop(0)
        if (row, col) in processed:
            continue
        processed.add((row, col))
        
        # Move down until hitting a splitter or exiting
        while row < len(grid):
            if grid[row][col] == '^':
                splitters_hit.add((row, col))
                # Create two new beams: left and right
                if col > 0:
                    beams.append((row + 1, col - 1))
                if col < len(grid[0]) - 1:
                    beams.append((row + 1, col + 1))
                break
            row += 1
    
    return len(splitters_hit)

# Test with example
with open('test.txt', 'r') as f:
    test_grid = [line.rstrip('\n') for line in f.readlines()]
test_start = test_grid[0].index('S')
test_result = solve_part1(test_grid, test_start)
print(f"Test result (expected 21): {test_result}")

# Solve for actual input
result_part1 = solve_part1(grid, start_col)
print(f"Part 1 answer: {result_part1}")

## Part 2: Quantum Beam Splitting

### Algorithm
In quantum interpretation, we track all possible paths a single particle can take. Each time it hits a splitter, it takes BOTH paths simultaneously. We need to count the number of unique ending positions (timelines).

Key insight: We need to track all possible paths and find where they end up. Each unique ending position represents a different timeline.

We'll use BFS/DFS to explore all possible paths from the starting position, tracking which ending positions are reachable.

In [ ]:
from functools import lru_cache

def solve_part2(grid, start_col):
    """Count the total number of distinct paths (timelines) through the grid."""
    
    @lru_cache(maxsize=None)
    def count_paths(row, col):
        # Out of bounds horizontally
        if col < 0 or col >= len(grid[0]):
            return 0
        
        # Reached bottom - this is one complete timeline
        if row >= len(grid):
            return 1
        
        # Check current position
        if grid[row][col] == '^':
            # Hit a splitter - particle takes both paths
            return count_paths(row + 1, col - 1) + count_paths(row + 1, col + 1)
        else:
            # Empty space - continue down
            return count_paths(row + 1, col)
    
    return count_paths(1, start_col)

# Test with example
test_result2 = solve_part2(tuple(test_grid), test_start)
print(f"Test result (expected 40): {test_result2}")

# Solve for actual input
result_part2 = solve_part2(tuple(grid), start_col)
print(f"Part 2 answer: {result_part2}")

## Summary

### Part 1: Classical Beam Splitting
Count the number of unique splitters that get hit by tachyon beams as they travel through the manifold.

**Answer: 1619**

### Part 2: Quantum Tachyon Splitting  
Count the total number of distinct paths (timelines) through the manifold. Each time a particle hits a splitter, it takes both paths simultaneously, creating multiple timelines.

**Answer: 23607984027985**

### Key Insights
- **Part 1**: Multiple beams can converge on the same splitter, so we count unique splitters hit, not total split events.
- **Part 2**: Use memoization to efficiently count all possible paths through the grid, as the number of paths grows exponentially.